# PyPort Basic Usage

This notebook demonstrates the basic usage of the PyPort library for interacting with the Port API.

## Setup

First, we need to install the PyPort library if it's not already installed:

In [ ]:
# Uncomment and run this cell if you need to install PyPort
# !pip install pyport

Next, we'll import the necessary modules and set up our API credentials:

In [ ]:
import os
import json
from typing import Dict, Any
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# Import PyPort
from pyport import PortClient
from pyport.exceptions import PortApiError

# Helper function to pretty-print JSON
def print_json(data: Dict[str, Any]) -> None:
    """Print JSON data in a readable format."""
    print(json.dumps(data, indent=2))

Set your Port API credentials. You can either set them as environment variables or directly in this notebook:

In [ ]:
# Option 1: Get credentials from environment variables
client_id = os.environ.get('PORT_CLIENT_ID')
client_secret = os.environ.get('PORT_CLIENT_SECRET')

# Option 2: Set credentials directly (not recommended for shared notebooks)
# client_id = "your-client-id"
# client_secret = "your-client-secret"

# Check if credentials are set
if not client_id or not client_secret:
    print("Warning: PORT_CLIENT_ID and PORT_CLIENT_SECRET environment variables are not set.")
    print("Please set these variables or provide them directly in the notebook.")
else:
    print("✅ API credentials are set!")

## Initializing the Client

Now, let's initialize the Port client:

In [ ]:
# Initialize the Port client
client = PortClient(
    client_id=client_id,
    client_secret=client_secret
)

print("✅ Client initialized successfully!")

## Getting Blueprints

Let's get all blueprints from your Port instance:

In [ ]:
try:
    blueprints = client.blueprints.get_blueprints()
    print(f"✅ Retrieved {len(blueprints)} blueprints!")
    
    # Convert to DataFrame for better visualization
    blueprint_data = []
    for bp in blueprints:
        blueprint_data.append({
            "identifier": bp.get("identifier"),
            "title": bp.get("title"),
            "description": bp.get("description", "")[:50] + "..." if bp.get("description", "") else "",
            "created_at": bp.get("createdAt"),
            "updated_at": bp.get("updatedAt")
        })
    
    # Create and display DataFrame
    blueprint_df = pd.DataFrame(blueprint_data)
    display(blueprint_df)
    
    # Store the first blueprint identifier for later use
    first_blueprint_id = blueprints[0]["identifier"] if blueprints else None
except PortApiError as e:
    print(f"❌ Error retrieving blueprints: {e}")

## Getting Blueprint Details

Now, let's get details for a specific blueprint:

In [ ]:
if first_blueprint_id:
    try:
        blueprint = client.blueprints.get_blueprint(first_blueprint_id)
        print(f"✅ Retrieved details for blueprint '{blueprint['title']}' ({blueprint['identifier']})!")
        
        # Display blueprint properties
        print("\nBlueprint Properties:")
        if "schema" in blueprint and "properties" in blueprint["schema"]:
            properties = blueprint["schema"]["properties"]
            property_data = []
            for prop_name, prop_details in properties.items():
                property_data.append({
                    "name": prop_name,
                    "title": prop_details.get("title", prop_name),
                    "type": prop_details.get("type", "unknown"),
                    "required": prop_name in blueprint["schema"].get("required", [])
                })
            
            # Create and display DataFrame
            property_df = pd.DataFrame(property_data)
            display(property_df)
        else:
            print("No properties defined for this blueprint.")
    except PortApiError as e:
        print(f"❌ Error retrieving blueprint details: {e}")
else:
    print("No blueprints available to get details for.")

## Getting Entities

Let's get all entities for a specific blueprint:

In [ ]:
if first_blueprint_id:
    try:
        entities = client.entities.get_entities(first_blueprint_id)
        print(f"✅ Retrieved {len(entities)} entities for blueprint '{first_blueprint_id}'!")
        
        # Convert to DataFrame for better visualization
        entity_data = []
        for entity in entities:
            entity_data.append({
                "identifier": entity.get("identifier"),
                "title": entity.get("title"),
                "created_at": entity.get("createdAt"),
                "updated_at": entity.get("updatedAt")
            })
        
        # Create and display DataFrame
        if entity_data:
            entity_df = pd.DataFrame(entity_data)
            display(entity_df)
        else:
            print("No entities found for this blueprint.")
        
        # Store the first entity identifier for later use
        first_entity_id = entities[0]["identifier"] if entities else None
    except PortApiError as e:
        print(f"❌ Error retrieving entities: {e}")
else:
    print("No blueprints available to get entities for.")

## Getting Entity Details

Now, let's get details for a specific entity:

In [ ]:
if first_blueprint_id and first_entity_id:
    try:
        entity = client.entities.get_entity(first_blueprint_id, first_entity_id)
        print(f"✅ Retrieved details for entity '{entity['title']}' ({entity['identifier']})!")
        
        # Display entity properties
        print("\nEntity Properties:")
        if "properties" in entity:
            properties = entity["properties"]
            property_data = []
            for prop_name, prop_value in properties.items():
                property_data.append({
                    "name": prop_name,
                    "value": str(prop_value) if not isinstance(prop_value, (dict, list)) else json.dumps(prop_value)
                })
            
            # Create and display DataFrame
            property_df = pd.DataFrame(property_data)
            display(property_df)
        else:
            print("No properties defined for this entity.")
    except PortApiError as e:
        print(f"❌ Error retrieving entity details: {e}")
else:
    print("No entities available to get details for.")

## Getting Actions

Let's get all actions from your Port instance:

In [ ]:
try:
    actions = client.actions.get_actions()
    print(f"✅ Retrieved {len(actions)} actions!")
    
    # Convert to DataFrame for better visualization
    action_data = []
    for action in actions:
        action_data.append({
            "identifier": action.get("identifier"),
            "title": action.get("title"),
            "trigger_type": action.get("trigger", {}).get("type"),
            "invocation_type": action.get("invocationMethod", {}).get("type"),
            "created_at": action.get("createdAt")
        })
    
    # Create and display DataFrame
    if action_data:
        action_df = pd.DataFrame(action_data)
        display(action_df)
    else:
        print("No actions found.")
except PortApiError as e:
    print(f"❌ Error retrieving actions: {e}")

## Visualizing Blueprint Distribution

Let's create a visualization of the entity distribution across blueprints:

In [ ]:
try:
    # Get all blueprints
    blueprints = client.blueprints.get_blueprints()
    
    # Count entities for each blueprint
    blueprint_counts = []
    for bp in blueprints:
        bp_id = bp["identifier"]
        try:
            entities = client.entities.get_entities(bp_id)
            blueprint_counts.append({
                "blueprint": bp["title"],
                "count": len(entities)
            })
        except PortApiError:
            # Skip if we can't get entities for this blueprint
            pass
    
    # Create DataFrame
    count_df = pd.DataFrame(blueprint_counts)
    
    # Create visualization
    plt.figure(figsize=(10, 6))
    plt.bar(count_df["blueprint"], count_df["count"])
    plt.xlabel("Blueprint")
    plt.ylabel("Number of Entities")
    plt.title("Entity Distribution Across Blueprints")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()
except PortApiError as e:
    print(f"❌ Error creating visualization: {e}")

## Summary

In this notebook, we've demonstrated the basic usage of the PyPort library:

1. Initializing the client
2. Getting blueprints and their details
3. Getting entities and their details
4. Getting actions
5. Visualizing entity distribution across blueprints

These operations form the foundation for more advanced usage of the Port API through the PyPort library.